## Generate user dataset

In [ ]:
!pip install names
import names

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 789 kB 4.8 MB/s 
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803699 sha256=a62897de0836de4beebca811c0aab98595894e1988d89ca6547378ed1dfcd7cc
  Stored in directory: /root/.cache/pip/wheels/05/ea/68/92f6b0669e478af9b7c3c524520d03050089e034edcc775c2b
Successfully built names


In [ ]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds = random_second)

There are mainly 6 groups of food: Milk/yogurts, Breakfast cereals, fruits, proteins, vegetables and carbohydrates. We will first ask the users to tell the system 3~4 favorite food of each type.

In [ ]:
def generateSample(group_no, food_list):
  cereal_list = ['Baked Products', 'Breakfast Cereals']
  protein_list = ['Dairy and Egg Products', 'Beef Products', 'Pork Products', 'Finfish and Shellfish Products', 'Sausages and Luncheon Meats', 'Poultry Products', 'Lamb, Veal, and Game Products']
  carbo_list = ['Dairy and Egg Products', 'Cereal Grains and Pasta', 'Fruits and Fruit Juices', 'Vegetables and Vegetable Products', 'Baked Products']
  group_list = ['Dairy and Egg Products', cereal_list, 'Fruits and Fruit Juices', protein_list, 'Vegetables and Vegetable Products', carbo_list]
  sample = level1_df.sample()
  
  group = group_list[group_no]
  if group_no%2 == 0:
    sample = level1_df.loc[level1_df['FoodGroup']==group].reset_index(drop=True).sample()
  else:
    sample = level1_df.loc[level1_df['FoodGroup'].isin(group)].reset_index(drop=True).sample()
  test = sample.iloc[0]['ID'] in food_list
  while(test):
    sample = generateSample(group_no, food_list)
    test = sample.iloc[0]['ID'] in food_list
    
  return sample

In [ ]:
def generateExtraSample(food_list):
  sample = level1_df.sample()
  test = sample.iloc[0]['ID'] in food_list
  while(test):
    sample = level1_df.sample()
    test = sample.iloc[0]['ID'] in food_list
  return sample

In [ ]:
def generateUserName(gender, name_list):
  random_name = names.get_first_name(gender=gender)
  test = random_name in name_list
  while(test):
    random_name = names.get_first_name(gender=gender)
    test = random_name in name_list
  return random_name

In [ ]:
# RECORD_NO: tells how many records each user has
def generateUserLike(USER_NO, RECORD_NO):
  name_list = []
  gender_list = ['female', 'male']
  user_gender = []
  users_like = pd.DataFrame(columns=['name','gender','food ID','Descrip'])

  # there are 50 users
  for loop in range(USER_NO):
    random_gender = random.choice(gender_list)
    random_name = generateUserName(random_gender, name_list)
    name_list.append(random_name)
    user_gender.append(random_gender)
    ID_list = []
    # count will count how many record has been added
    count = 0
    count_test = False

    for group_no in range(0,6):
      if count >= RECORD_NO or count_test:
        break
      for i in range(random.randint(3,4)):
        sample = generateSample(group_no, ID_list)
        sample_id = sample.iloc[0]['ID']
        sample_name = sample.iloc[0]['Descrip']
        ID_list.append(sample_id)
        # store into users_like
        users_like.at[loop*RECORD_NO+count, 'name'] = random_name
        users_like.at[loop*RECORD_NO+count, 'gender'] = random_gender
        users_like.at[loop*RECORD_NO+count, 'food ID'] = sample_id
        users_like.at[loop*RECORD_NO+count, 'Descrip'] = sample_name
        count += 1
        if count >= RECORD_NO:
          count_test = True
          break
    
    
    while (count < RECORD_NO):
      sample = generateExtraSample(ID_list)
      sample_id = sample.iloc[0]['ID']
      sample_name = sample.iloc[0]['Descrip']
      ID_list.append(sample_id)
      users_like.at[loop*RECORD_NO+count, 'name'] = random_name
      users_like.at[loop*RECORD_NO+count, 'gender'] = random_gender
      users_like.at[loop*RECORD_NO+count, 'food ID'] = sample_id
      users_like.at[loop*RECORD_NO+count, 'Descrip'] = sample_name
      count += 1
      
  return name_list, user_gender, users_like

In [ ]:
def compete(like, sample, weight):
  sample_id = sample.iloc[0]['ID']
  id_list = like['food ID'].unique().tolist()
  if sample_id in id_list:
    return sample
  else:
    id_list.append(sample_id)
    size = len(id_list)
    weight_list = []
    for i in range(size-1):
      weight_list.append(10)
    weight_list.append(weight)
    weight_list = tuple(weight_list)
    choice_list = random.choices(
        id_list, weights=weight_list, k=1)
    choice_id = choice_list[0]
    new_sample = level1_df.loc[level1_df['ID'] == choice_id]
    return new_sample

In [ ]:
users = pd.DataFrame(columns=['name','gender','height_cm','weight_kg','age','food ID','Descrip','last eat timestamp','frequency_day'])
RECORD_NO = 100

start = datetime.strptime('3/1/2022 7:30 AM', '%m/%d/%Y %I:%M %p')
end = datetime.strptime('3/8/2022 11:00 PM', '%m/%d/%Y %I:%M %p')
current = datetime.strptime('3/10/2022 4:00 PM', '%m/%d/%Y %I:%M %p')
base = datetime.strptime('3/1/2022 7:30 AM', '%m/%d/%Y %I:%M %p')

# generate 50 users, each of them has 20 favourite food
user_name, user_gender, user_like = generateUserLike(50, 20)

for loop in range(50):
  # food_record is a list to record (ID, day) so that there will not be same food record in same day!
  food_record = []
  random_gender = user_gender[loop]
  random_name = user_name[loop]
  random_like = user_like.loc[user_like['name'] == random_name]
  if random_gender=='female':
    random_height = random.randint(150,175)
    random_weight = random.randint(40,70)
  else:
    random_height = random.randint(160,185)
    random_weight = random.randint(50,80)
  random_age = random.randint(18,50)
  for count in range(RECORD_NO):
    sample = level1_df.sample()

    choice = compete(random_like, sample, 50)

    sample_id = choice.iloc[0]['ID']
    sample_name = choice.iloc[0]['Descrip']
    sample_date = random_date(start, end)

    test = (sample_id, sample_date.date()) in food_record
    t = 0

    while(test):

      # if iterate over 10 times, consider change sample
      if (t>=5):
        sample = level1_df.sample()
        choice = compete(random_like, sample, 50)

        sample_id = choice.iloc[0]['ID']
        sample_name = choice.iloc[0]['Descrip']
      
      sample_date = random_date(start, end)
      test = (sample_id, sample_date.date()) in food_record
      t += 1
    
    food_record.append((sample_id, sample_date.date()))
    
    users.at[loop*RECORD_NO+count, 'name'] = random_name
    users.at[loop*RECORD_NO+count, 'gender'] = random_gender
    users.at[loop*RECORD_NO+count, 'height_cm'] = random_height
    users.at[loop*RECORD_NO+count, 'weight_kg'] = random_weight
    users.at[loop*RECORD_NO+count, 'age'] = random_age
    users.at[loop*RECORD_NO+count, 'food ID'] = sample_id
    users.at[loop*RECORD_NO+count, 'Descrip'] = sample_name
    users.at[loop*RECORD_NO+count, 'last eat timestamp'] = sample_date
    users.at[loop*RECORD_NO+count, 'frequency_day'] = random.randint(1,3)


In [ ]:
users

,name,gender,height_cm,weight_kg,age,food ID,Descrip,last eat timestamp,frequency_day
0,Tonya,female,154,46,48,8449.0,"Cereals, QUAKER, Instant Grits Product with Am...",2022-03-08 19:59:24,3
1,Tonya,female,154,46,48,15017.0,"Fish, cod, Atlantic, canned, solids and liquid",2022-03-02 19:08:40,3
2,Tonya,female,154,46,48,11096.0,"Broccoli raab, raw",2022-03-07 14:06:08,3
3,Tonya,female,154,46,48,9434.0,"Guanabana nectar, canned",2022-03-06 08:46:57,3
4,Tonya,female,154,46,48,35154.0,"Salmonberries, raw (Alaska Native)",2022-03-06 19:49:59,1
...,...,...,...,...,...,...,...,...,...
4995,Katherine,female,153,64,47,11497.0,"Succotash, (corn and limas), canned, with crea...",2022-03-04 10:44:12,1
4996,Katherine,female,153,64,47,11827.0,"Pokeberry shoots, (poke), cooked, boiled, drai...",2022-03-02 08:50:10,2
4997,Katherine,female,153,64,47,7205.0,"OSCAR MAYER, Bologna Light (pork, chicken, beef)",2022-03-08 20:08:19,1
4998,Katherine,female,153,64,47,11770.0,"Corn, sweet, yellow, cooked, boiled, drained, ...",2022-03-03 02:20:59,3


user_like is a dataframe recording top 20 favorite food of each user.

In [ ]:
user_like

,name,gender,food ID,Descrip
0,Tonya,female,1077.0,"Milk, whole, 3.25% milkfat, with added vitamin D"
1,Tonya,female,1070.0,"Dessert topping, powdered"
2,Tonya,female,43261.0,"Yogurt, fruit variety, nonfat"
3,Tonya,female,18567.0,"KRAFT, STOVE TOP Stuffing Mix Chicken Flavor"
4,Tonya,female,18630.0,"PILLSBURY, Chocolate Chip Cookies, refrigerate..."
...,...,...,...,...
995,Katherine,female,11487.0,"Squash, winter, butternut, frozen, unprepared"
996,Katherine,female,11096.0,"Broccoli raab, raw"
997,Katherine,female,11497.0,"Succotash, (corn and limas), canned, with crea..."
998,Katherine,female,11770.0,"Corn, sweet, yellow, cooked, boiled, drained, ..."
